# NLP - HW2 - Parte2A: Entrenamiento TfIdf

Con el objetivo de aplicar los conocimientos presentados en clase y a su vez generar los modelos solicitados, en este punto se hará uso de la herramienta Databricks y Pyspark con el objetivo de leer los archivos, generar la construcción del dataset y hacer el entrenamiento respectivo de los modelos.

Para la elaboración de la actividad se sigue una serie de pasos:

* Se crea un cluster de spark 3.1 en Databricks, con mínimo 1 driver, 1 nodo y autoescalable a máximo 20.
* Cada nodo es una instancia m4.large de AWS con 2 core y 8gb ram, el driver tiene la misma configuración.
* Los archivos se almacenan en un bucket de s3, se descomprime y se hace la lectura para la construcción del dataset.
* Se hace uso de spark para paralelizar los trabajos y hacerlos más eficiente a la hora de entrenar los modelos.

## Punto No. 2: Naive Bayes (NB) & Logistic Regression (LR)

### For the 20N dataset compare two classifiers NB and LR to identify the 20 different newsgroups.

In [4]:
# importar librerias

import pyspark.sql.functions as F
from pyspark.sql.types import *

# Librerias para procesar datos
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, StringIndexer,  HashingTF, IDF

# librerias sparknlp
import sparknlp
from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

# ml libraries
from pyspark.ml.classification import NaiveBayes, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [5]:
# Importar dataset procesado para la ejecución de los modelos
sdf = spark.read.format('parquet').load('/mnt/databricks-mine/HW02/news20processed')

In [6]:
# Creación del pipeline para hacer el modelo
indexer = StringIndexer(inputCol="fileName", outputCol="labelIndex")
hasher = HashingTF(inputCol='tokens', outputCol="hash")
idf = IDF(inputCol='hash', outputCol="features_tfidf")

# Definición del pipeline
pipeline = Pipeline(stages=[indexer, hasher, idf])

# fit y transform del pipeline
pipeData = pipeline.fit(sdf).transform(sdf)

In [7]:
pipeData.select("fileName","labelIndex").groupBy('fileName', 'labelIndex').count().orderBy('labelIndex').show(25)

+--------------------+----------+-----+
 fileName|labelIndex|count|
+--------------------+----------+-----+
 rec.sport.hockey| 0.0| 999|
soc.religion.chri...| 1.0| 997|
 rec.motorcycles| 2.0| 994|
 rec.sport.baseball| 3.0| 994|
 sci.crypt| 4.0| 991|
 rec.autos| 5.0| 990|
 sci.med| 6.0| 990|
 sci.space| 7.0| 987|
comp.os.ms-window...| 8.0| 985|
comp.sys.ibm.pc.h...| 9.0| 982|
 sci.electronics| 10.0| 981|
 comp.windows.x| 11.0| 980|
 comp.graphics| 12.0| 973|
 misc.forsale| 13.0| 972|
comp.sys.mac.hard...| 14.0| 961|
talk.politics.mid...| 15.0| 940|
 talk.politics.guns| 16.0| 910|
 alt.atheism| 17.0| 799|
 talk.politics.misc| 18.0| 775|
 talk.religion.misc| 19.0| 628|
+--------------------+----------+-----+

##### Divide the dataset into training (60%), development (10%) and test (30%)

In [9]:
model = pipeData.select('labelIndex', 'features_tfidf')
(trainData, testData, devData) = model.randomSplit([0.6, 0.3, 0.1], seed=321)

##### Train NB and LR using the following representations:

In [11]:
from pyspark.ml.classification import NaiveBayes, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

###### Personalized representation. You as a designer must define the select set of characteristics. Explain your feature selection strategy in detail.

Train tfidf selection strategy

In [14]:
# NaiveBayes
nbTf = NaiveBayes(modelType="multinomial",labelCol="labelIndex", featuresCol="features_tfidf")
nbModelTf = nbTf.fit(trainData).transform(testData)

In [15]:
# LogisticRegression
lrTf = LogisticRegression(labelCol="labelIndex", featuresCol="features_tfidf")
lrModelTf = lrTf.fit(trainData).transform(testData)

Test tfidf selection models

In [17]:
nbModelTf.select("prediction", "labelIndex", "features_tfidf").show(5)

+----------+----------+--------------------+
prediction|labelIndex| features_tfidf|
+----------+----------+--------------------+
 15.0| 15.0|(262144,[1,432,53...|
 15.0| 15.0|(262144,[56,619,1...|
 15.0| 15.0|(262144,[95,535,8...|
 15.0| 15.0|(262144,[154,1971...|
 15.0| 15.0|(262144,[181,535,...|
+----------+----------+--------------------+
only showing top 5 rows

In [18]:
lrModelTf.select("prediction", "labelIndex", "features_tfidf").show(5)

+----------+----------+--------------------+
prediction|labelIndex| features_tfidf|
+----------+----------+--------------------+
 15.0| 15.0|(262144,[1,432,53...|
 15.0| 15.0|(262144,[56,619,1...|
 15.0| 15.0|(262144,[95,535,8...|
 18.0| 15.0|(262144,[154,1971...|
 15.0| 15.0|(262144,[181,535,...|
+----------+----------+--------------------+
only showing top 5 rows

In [19]:
# Evaluación NaiveBayes
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nbModelTf)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 0.933145
Test Error of NaiveBayes = 0.0668553